## This file reads the review data and clean it, join with user data as well

In [15]:
import os 
import json 
import tqdm 

from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz

import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml

In [38]:
# file directory
file_business, file_review, file_user = [
    '..\\data\\business.json',
    '..\\data\\yelp_academic_dataset_review.json',
    '..\\data\\user.json',
]

### Read in review data 
df has 5996996 raw review histories

df_join has 5903215 review histories after selecting the ones that has user average star history

In [39]:
#path = "yelp_academic_dataset_review.json"

#Litos' way
#file = open(path, encoding="utf8")
#data = file.readlines()
#data = list(map(json.loads, data))

#Json's way
with open(file_review, encoding = 'utf-8') as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data))

df_review = pd.DataFrame(data)

In [83]:
data

[{'review_id': 'x7mDIiDB3jEiPGPHOmDzyw',
  'user_id': 'msQe1u7Z_XuqjGoqhB0J5g',
  'business_id': 'iCQpiavjjPzJ5_3gPD5Ebg',
  'stars': 2,
  'date': '2011-02-25',
  'text': "The pizza was okay. Not the best I've had. I prefer Biaggio's on Flamingo / Fort Apache. The chef there can make a MUCH better NY style pizza. The pizzeria @ Cosmo was over priced for the quality and lack of personality in the food. Biaggio's is a much better pick if youre going for italian - family owned, home made recipes, people that actually CARE if you like their food. You dont get that at a pizzeria in a casino. I dont care what you say...",
  'useful': 0,
  'funny': 0,
  'cool': 0},
 {'review_id': 'dDl8zu1vWPdKGihJrwQbpw',
  'user_id': 'msQe1u7Z_XuqjGoqhB0J5g',
  'business_id': 'pomGBqfbxcqPv14c3XH-ZQ',
  'stars': 5,
  'date': '2012-11-13',
  'text': 'I love this place! My fiance And I go here atleast once a week. The portions are huge! Food is amazing. I love their carne asada. They have great lunch specials.

In [46]:
df_review.head(2)  #5996995   review histories

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g


### Reading in user data 
total users: 1637138 

In [25]:
# loading review data to pandas
with open(file_user, encoding = 'utf-8') as f:
    df_user = pd.DataFrame(json.loads(line) for line in f)

In [69]:
#no user that has no average rating history
#df_user.average_stars.isna().sum()
df_user.average_stars.min()

1.0

In [71]:
df_user.loc[df_user.average_stars == '1.0']

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since


In [43]:
#Select the information we want from the users 
selected_features = ['user_id','average_stars']
df_user_selectedStars = df_user.filter(items = selected_features)

In [45]:
df_user_selectedStars.head(2)

,user_id,average_stars
0,l6BmjZMeQD3rDxWUbiAiow,4.03
1,4XChL029mKr5hydo79Ljxg,3.63


### Combining review data and user average rating data, inner join on user_id

In [47]:
# set user_id as the index of two data frames
df_user_selectedStars = df_user_selectedStars.set_index('user_id')
df_review = df_review.set_index('user_id')

In [49]:
df_review.head(2)

,business_id,cool,date,funny,review_id,stars,text,useful
user_id,,,,,,,,
msQe1u7Z_XuqjGoqhB0J5g,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0
msQe1u7Z_XuqjGoqhB0J5g,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0


In [48]:
df_user_selectedStars.head(2)

,average_stars
user_id,
l6BmjZMeQD3rDxWUbiAiow,4.03
4XChL029mKr5hydo79Ljxg,3.63


In [50]:
# combine two dataset with the same user id 

df_join = df_review.join(df_user_selectedStars,how = 'inner')
df_join.reset_index(inplace = True)

In [53]:
df_join.head(2)

,user_id,business_id,cool,date,funny,review_id,stars,text,useful,user_avg_stars
0,---1lKK3aKOuomHnwAkAow,D1PhUlkQA1ZsVe9Cx4yqOw,0,2011-12-11,1,iX0tGIeZiVwyQYFaGXXAIA,5,I wish I had known more about this when I was ...,1,3.92
1,---1lKK3aKOuomHnwAkAow,CWNMLT-ppaUjLMmrnYDPVg,2,2010-11-05,3,_U_ctMQkF3Smqo9mjpw1Iw,5,Why the hell does anyone go to Kona Grill? Un...,3,3.92


In [54]:
# rename the column name "stars" to "avg_stars" in business dataset to aviod nameing conflicts with review dataset
df_join = df_join.rename(index = str, columns = {"average_stars":"user_avg_stars"})

In [72]:
df_join.head(2)

,user_id,business_id,cool,date,funny,review_id,stars,text,useful,user_avg_stars
0,---1lKK3aKOuomHnwAkAow,D1PhUlkQA1ZsVe9Cx4yqOw,0,2011-12-11,1,iX0tGIeZiVwyQYFaGXXAIA,5,I wish I had known more about this when I was ...,1,3.92
1,---1lKK3aKOuomHnwAkAow,CWNMLT-ppaUjLMmrnYDPVg,2,2010-11-05,3,_U_ctMQkF3Smqo9mjpw1Iw,5,Why the hell does anyone go to Kona Grill? Un...,3,3.92


In [82]:
#df_join.to_json('reviewNuserAvgFile.json',orient='records', lines=True)

In [89]:
#data_hasUserAvg = df_join.to_json(orient='records', lines=True)
df_join.size

59032150

In [90]:
#df_join.to_json(r'reviewWuserAvg.json',orient='split')

### Read in business data
192608 raw business

57161 valid businesses for use 

In [140]:
# loading business data to pandas
with open(file_business, encoding = 'utf-8') as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

MemoryError: 

In [78]:
df_business.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON


In [118]:
df_business.shape[0]

192609

In [138]:
(df_business['is_open'] == 1).sum()

158525

In [ ]:
# remove closed business
df_business = df_business[df_business['is_open'] == 1]

In [139]:
df_business.shape[0]

192609

In [92]:
#Only select certain features to join 
selected_features_for_join = ['business_id','stars','categories','attributes']
df_business_forJoin = df_business.filter(items = selected_features_for_join)

In [96]:
df_business_forJoin = df_business_forJoin.rename(index = str, columns = {"stars": "business_stars"})
df_business_forJoin.head(2)

,business_id,business_stars,categories,attributes
0,1SWheh84yJXfytovILXOAQ,3.0,"Golf, Active Life",{'GoodForKids': 'False'}
1,QXAEGFB4oINsVuTFxEYKFQ,2.5,"Specialty Food, Restaurants, Dim Sum, Imported...","{'RestaurantsReservations': 'True', 'GoodForMe..."


In [132]:
df_business_forJoin.shape[0]

57161

In [131]:
pd.set_option('display.max_colwidth', -1)
df_business_forJoin.categories

1         Specialty Food, Restaurants, Dim Sum, Imported Food, Food, Chinese, Ethnic Food, Seafood                                                                 
2         Sushi Bars, Restaurants, Japanese                                                                                                                        
11        Restaurants, Breakfast & Brunch, Mexican, Tacos, Tex-Mex, Fast Food                                                                                      
13        Italian, Restaurants, Pizza, Chicken Wings                                                                                                               
17        Restaurants, Italian                                                                                                                                     
23        Sandwiches, Salad, Restaurants, Burgers, Comfort Food                                                                                                    
25        Mexica

In [97]:
# remove null row for categories column and attributes column
cond_category_not_null = ~df_business_forJoin['categories'].isnull()
cond_attribute_not_null = ~df_business_forJoin['attributes'].isnull()
df_business_forJoin = df_business_forJoin[cond_category_not_null & cond_attribute_not_null]

In [123]:
cond_category_not_null.sum()

192127

In [100]:
attributeSet = []
for attribute in df_business_forJoin['attributes']:
    for (key, item) in attribute.items():
        attributeSet.append(key)

In [101]:
arr = np.array(attributeSet)
np.unique(arr)

array(['AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'Ambience', 'BYOB',
       'BYOBCorkage', 'BestNights', 'BikeParking',
       'BusinessAcceptsBitcoin', 'BusinessAcceptsCreditCards',
       'BusinessParking', 'ByAppointmentOnly', 'Caters', 'CoatCheck',
       'Corkage', 'DietaryRestrictions', 'DogsAllowed', 'DriveThru',
       'GoodForDancing', 'GoodForKids', 'GoodForMeal',
       'HairSpecializesIn', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi'],
      dtype='<U26')

In [106]:
cateSet = []
for attribute in df_business_forJoin['categories']:
        attributeSet.append(attribute)

arr = np.array(cateSet)
np.unique(arr)

array([], dtype=float64)

In [127]:
# We only take restaurant business
cond_category_restaurant = df_business_forJoin["categories"].str.contains("Restaurants|restaurant",case=True)
df_business_forJoin = df_business_forJoin[cond_category_restaurant]

In [128]:
df_business_forJoin.shape[0]

57161

#### Clean the business data
filter out the businesses that are closed, and not restaurants, and probably that don't have attributes/categories

### The way to split the data into smaller portion

In [ ]:
total_len = len(data)
num = 10
chunk_len = total_len // num

folder = "data"
if not os.path.exists(folder):
    os.makedirs(folder)
for i in range(num):
    name = os.path.join(folder, "%d.txt" % i) 
    with open(name, 'a') as f:
        for line in tqdm.tqdm(data[i*chunk_len :  (i+1)*chunk_len]):
            data_s = json.dumps(line)+"\n"
            f.write(data_s)
    print("fininshed file: ", i)

name = os.path.join(folder, "%d.txt" % num) 
with open(name, 'a') as f:
    for line in tqdm.tqdm(data[num*chunk_len :  total_len]):
        data_s = json.dumps(line)+"\n"
        f.write(data_s)
print("fininshed file: ", i)